# Pluto and Eris base fields.

This notebooks computes all the constants necessary for the implementation of Pluto and Eris base fields in halo2curves.<br>
Supporting evidence for this half-pairing cycle <https://github.com/daira/pluto-eris/tree/main><br>
Ark-works implementation <https://github.com/arkworks-rs/curves/pull/54> (Notice that some constants are different due to the different choice of generator.)<br>

In [1]:
# Compute R, R2, R3
def compute_rs(F):
    r = F(2) ** 448
    r2 = r **2
    r3 = r**3
    return (r, r2, r3)

# Compute s, t
def compute_s(r):
    aux = r -1
    for i in range(446):
        if aux %2:
            return i 
        aux = aux/2

def compute_t(r, s):
    return (r-1)/ 2**s

def generator(F):
    return F.primitive_element()

def compute_root_of_unity(F, gen, t):
    return gen**t

def compute_delta(F, gen, s):
    return gen**(2**s)

def compute_zetas(F):
    zeta = F.zeta(3)
    zeta2 = zeta**2
    return zeta, zeta2

def compute_inv(mod):
    Z = IntegerModRing(2^64)
    p = Z(mod)
    return -(p^(-1))


In [2]:
def get_all_constants(mod):
    F = GF(mod)
    r, r2, r3 = compute_rs(F)
    neg_one = F(1) * r

    s = compute_s(mod)
    t = compute_t(mod, s)

    gen = generator(F)
    
    root_of_unity = compute_root_of_unity(F, gen, t)
    root_of_unity_inv = root_of_unity.inverse()
    delta = compute_delta(F, gen, s)
    two_inv = F(2).inverse()

    inv_const = mod - 2
    t_minus_1_over_2 = (t-1)/2

    inv = compute_inv(mod)
    zeta, zeta2 = compute_zetas(F)

    names = ["Mod", "NEG_ONE", "R", "R2", "R3", "S", "T", "Gen", "Root of unity", "Root of unity inverse", "Delta", "Two inv", "inv const(p-2)", "(T-1)/2", "Inv", "Zeta", "Zeta2"]
    all_constants = [mod, neg_one, r, r2, r3, s, t, gen, root_of_unity, root_of_unity_inv, delta, two_inv, inv_const, t_minus_1_over_2, inv, zeta, zeta2]
    constants_dict = {}
    for name, c in zip(names, all_constants):
        constants_dict[name] = c
        print(name)
        print_for_rust(c)

    return constants_dict
    

In [3]:
# Helper functions
def u_64_little_endian(n):
    str_hex = hex(n)
    str_hex_without_0x = str_hex[2:]
    full_width_str = '0' * (112 - len(str_hex_without_0x)) + str_hex_without_0x
    assert len(full_width_str) == 112

    res = []
    for i in range(7):
        temp = '0x' + full_width_str[112 - 16 * (i + 1) : 112 - 16 * i]
        res.append(temp)
    return res
# This is the format used in halo2curves, a 7 element slice of u64.
def print_for_rust(elem):
    print( hex(elem))
    print('\n')
    print('[')
    for limb in u_64_little_endian(elem):
        print(limb + ",")
    print(']')
    print('------\n')

In [4]:
# Pluto base field
p = 0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5cda8a6c7be4a7a5fe8fadffd6a2a7e8c30006b9459ffffcd300000001
fp_constants = get_all_constants(p)

Mod
0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5cda8a6c7be4a7a5fe8fadffd6a2a7e8c30006b9459ffffcd300000001


[
0x9ffffcd300000001,
0xa2a7e8c30006b945,
0xe4a7a5fe8fadffd6,
0x443f9a5cda8a6c7b,
0xa803ca76f439266f,
0x0130e0000d7f70e4,
0x2400000000002400,
]
------

NEG_ONE
0x3ffffffffff03fff7a9dfffa183e9bf67e576bf526ff2f52242c7760637089cbf6a760a123e01218d68a2aaffd0ef18a000163afffffff9


[
0xa000163afffffff9,
0x8d68a2aaffd0ef18,
0xbf6a760a123e0121,
0x2242c7760637089c,
0x67e576bf526ff2f5,
0xf7a9dfffa183e9bf,
0x03ffffffffff03ff,
]
------

R
0x3ffffffffff03fff7a9dfffa183e9bf67e576bf526ff2f52242c7760637089cbf6a760a123e01218d68a2aaffd0ef18a000163afffffff9


[
0xa000163afffffff9,
0x8d68a2aaffd0ef18,
0xbf6a760a123e0121,
0x2242c7760637089c,
0x67e576bf526ff2f5,
0xf7a9dfffa183e9bf,
0x03ffffffffff03ff,
]
------

R2
0x1a4b16581f66e3cc8bcb0f20758aec8520b6db3d7481a84c734fd363b575c23e7a42067a8ccd154b4b20c07277ae01f1d9702c6d54dc0598


[
0xd9702c6d54dc0598,
0x4b20c07277ae01f1,
0x7a42067a8ccd154b,

In [5]:
# Pluto scalar field
q = 0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5c7a8a6c7be4a775fe8e177fd69ca7e85d60050af41ffffcd300000001
fq_constants = get_all_constants(q)

Mod
0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5c7a8a6c7be4a775fe8e177fd69ca7e85d60050af41ffffcd300000001


[
0x1ffffcd300000001,
0x9ca7e85d60050af4,
0xe4a775fe8e177fd6,
0x443f9a5c7a8a6c7b,
0xa803ca76f439266f,
0x0130e0000d7f70e4,
0x2400000000002400,
]
------

NEG_ONE
0x3ffffffffff03fff7a9dfffa183e9bf67e576bf526ff2f52242c778a637089cbf6bc60a1d5b8121b768a5725fdcb3532000163afffffff9


[
0x2000163afffffff9,
0xb768a5725fdcb353,
0xbf6bc60a1d5b8121,
0x2242c778a637089c,
0x67e576bf526ff2f5,
0xf7a9dfffa183e9bf,
0x03ffffffffff03ff,
]
------

R
0x3ffffffffff03fff7a9dfffa183e9bf67e576bf526ff2f52242c778a637089cbf6bc60a1d5b8121b768a5725fdcb3532000163afffffff9


[
0x2000163afffffff9,
0xb768a5725fdcb353,
0xbf6bc60a1d5b8121,
0x2242c778a637089c,
0x67e576bf526ff2f5,
0xf7a9dfffa183e9bf,
0x03ffffffffff03ff,
]
------

R2
0x50d7c998f46144ee436895a5a630ff544d51e923f64695651da4da1c97f716419bd905e6e4ff6c2bc64e865fe4552ad740808c831022522


[
0x740808c831022522,
0xbc64e865fe4552ad,
0x19bd905e6e4ff6c2,
